### The Data
This book will try to develop an algorithm to identify an atoms nearby functional groups, that is, in CH3-CH2-OH the H atoms in CH3 are part of a CH3-group, neigherst neighbor are CH2 followed by OH.

Note: this book will only consider data from the following csv files: train, test and structures

In [1]:
import pandas as pd

input_folder = './input'

train = pd.read_csv(f'{input_folder}/train.csv')
structures = pd.read_csv(f'{input_folder}/structures.csv')

In [2]:
f'Train shape: {train.shape}'

'Train shape: (4658147, 6)'

In [3]:
f'Structures shape: {structures.shape}'

'Structures shape: (2358657, 6)'

In [4]:
print(f"There are {train['type'].nunique()} unique coupling types: {train['type'].unique()}")


There are 8 unique coupling types: ['1JHC' '2JHH' '1JHN' '2JHN' '2JHC' '3JHH' '3JHC' '3JHN']


#### Visualising a few molecules from train, structures and structured/xyz-files

In [5]:
train.loc[train['molecule_name'] == 'dsgdb9nsd_000013']

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
106,106,dsgdb9nsd_000013,3,0,1JHC,83.51320
107,107,dsgdb9nsd_000013,3,1,2JHC,-2.03004
108,108,dsgdb9nsd_000013,3,2,3JHC,2.51277
109,109,dsgdb9nsd_000013,3,4,2JHH,-11.43170
110,110,dsgdb9nsd_000013,3,5,2JHH,-11.47950
111,111,dsgdb9nsd_000013,3,6,3JHH,3.42929
112,112,dsgdb9nsd_000013,3,7,3JHH,12.33070
113,113,dsgdb9nsd_000013,4,0,1JHC,83.51430
114,114,dsgdb9nsd_000013,4,1,2JHC,-2.03044
115,115,dsgdb9nsd_000013,4,2,3JHC,2.51802


In [6]:
structures.loc[structures['molecule_name'] == 'dsgdb9nsd_000230']

,molecule_name,atom_index,atom,x,y,z
2365,dsgdb9nsd_000230,0,C,0.087038,1.394377,0.008724
2366,dsgdb9nsd_000230,1,O,-0.030017,-0.008995,0.002400
2367,dsgdb9nsd_000230,2,C,-1.350280,-0.574676,0.011941
2368,dsgdb9nsd_000230,3,C,-2.137122,-0.166239,-1.244013
2369,dsgdb9nsd_000230,4,C,-1.088529,-2.084449,0.001286
2370,dsgdb9nsd_000230,5,C,-2.112947,-0.179915,1.287035
2371,dsgdb9nsd_000230,6,H,1.158248,1.613943,-0.000143
2372,dsgdb9nsd_000230,7,H,-0.365542,1.865800,-0.875376
2373,dsgdb9nsd_000230,8,H,-0.348864,1.856390,0.906072
2374,dsgdb9nsd_000230,9,H,-1.558715,-0.392910,-2.144716


### Working with molecules using Pybel and  Open Babel


#### Finding the molecular smiles

In [7]:
import pybel

def xyz_to_smiles(fname):    
    mol = next(pybel.readfile("xyz", fname))
    smi = mol.write(format="smi")
    return smi.split()[0].strip()

In [8]:
smi = xyz_to_smiles("./input/structures/dsgdb9nsd_000050.xyz")
print(smi)

[nH]1cccc1


#### Finding functional groups

In [9]:
# Finding atomic indexes of -CH3 and -O- groups

mol = next(pybel.readfile("xyz", "./input/structures/dsgdb9nsd_000230.xyz"))

smarts_CH3 = pybel.Smarts("[CH3]") 
smarts_OH0 = pybel.Smarts("[OH0]") 
print(smarts_CH3.findall(mol))
print(smarts_OH0.findall(mol))

print(len(mol.atoms))

[(1,), (4,), (5,), (6,)]
[(2,)]
18


### Functional Groups and other features
#### List of Functional Groups we will classify by

In [10]:
import openbabel

def describe_molecule(name):
    
    mol = next(pybel.readfile("xyz", "./input/structures/" + name + ".xyz"))
    mol_length = mol.OBMol.NumBonds()

    print(f'Molecule length: {mol_length}')
    # Define containers
    mol_name = [0] * mol_length # This will be our dataframe index
    atom_0 = [0] * mol_length
    atom_1 = [0] * mol_length
    bond_order = [0] * mol_length
    bond_length = [0] * mol_length
    is_in_ring = [0] * mol_length
    has_aromatic_bond = [0] * mol_length
    is_primary_amide = [0] * mol_length
    is_secondary_amide = [0] * mol_length
    is_tertiary_amide = [0] * mol_length
    is_ester = [0] * mol_length
    is_carbonyl = [0] * mol_length

    atom_0_CH3 = [0] * mol_length
    atom_0_CH2= [0] * mol_length
    atom_0_CH1= [0] * mol_length
    atom_0_CH0= [0] * mol_length
    atom_1_CH3 = [0] * mol_length
    atom_1_CH2= [0] * mol_length
    atom_1_CH1= [0] * mol_length
    atom_1_CH0= [0] * mol_length

    atom_0_NH2 = [0] * mol_length
    atom_0_NH1= [0] * mol_length
    atom_0_NH0= [0] * mol_length
    atom_1_NH2 = [0] * mol_length
    atom_1_NH1= [0] * mol_length
    atom_1_NH0= [0] * mol_length

    atom_0_OH1 = [0] * mol_length
    atom_0_OH0= [0] * mol_length
    atom_1_OH1 = [0] * mol_length
    atom_1_OH0= [0] * mol_length

    bonds_considered = {}
    index = -1
    for atom in mol.atoms:

        if atom.atomicnum == 1: # ignore H atoms
            continue

        atom_id = atom.idx
        print("Atom index {}".format(atom_id))
        for neighbour_atom in openbabel.OBAtomAtomIter(atom.OBAtom):

            neighbour_atom_id = neighbour_atom.GetIdx()

            key = str(min(atom_id, neighbour_atom_id)) + '_' + str(max(atom_id, neighbour_atom_id))
            if key in bonds_considered:
                continue
            else:
                bonds_considered[key] = 'counted'

            index += 1
            print(f'Index: {index}')


            print("  atom index:atom {}: {}, Neighbor index:atom {}: {}".format(atom_id, atom.atomicnum, neighbour_atom_id, neighbour_atom.GetAtomicNum()))
            bond = mol.OBMol.GetBond(atom.idx, neighbour_atom.GetIdx())

            mol_name[index] = name
            atom_0[index] = neighbour_atom_id - 1
            atom_1[index] = atom_id - 1

            if atom.atomicnum == 6:
                if neighbour_atom.GetAtomicNum() == 1:
                    if bond.IsAromatic():
                        atom_0_CH1[index] = 1

                        atom_1_CH1[index] = 1
                    else:
                        atom_0_CH3[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() >= 3 else 0 # > handle CH4
                        atom_0_CH2[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() == 2 else 0
                        atom_0_CH1[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() == 1 else 0
                        atom_0_CH0[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() == 0 else 0

                        atom_1_CH3[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() >= 3 else 0 # > handle CH4
                        atom_1_CH2[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() == 2 else 0
                        atom_1_CH1[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() == 1 else 0
                        atom_1_CH0[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() == 0 else 0
                else:
                    if bond.IsAromatic():
                        if neighbour_atom.GetAtomicNum() == 6:
                            atom_0_CH1[index] = 1

                        elif neighbour_atom.GetAtomicNum() == 7:
                            atom_0_NH1[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 1 else 0
                            atom_0_NH0[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 0 else 0

                        elif neighbour_atom.GetAtomicNum() == 8:
                            atom_0_OH0[index] =1

                        atom_1_CH1[index] = 1
                    else:
                        if neighbour_atom.GetAtomicNum() == 6:
                            atom_0_CH3[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 3 else 0
                            atom_0_CH2[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 2 else 0
                            atom_0_CH1[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 1 else 0
                            atom_0_CH0[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 0 else 0
                        elif neighbour_atom.GetAtomicNum() == 7:
                            atom_0_NH2[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 2 else 0
                            atom_0_NH1[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 1 else 0
                            atom_0_NH0[index] =1 if neighbour_atom.ExplicitHydrogenCount() == 0 else 0
                        elif neighbour_atom.GetAtomicNum() == 8:
                            atom_0_OH1[index] =1 if neighbour_atom.ExplicitHydrogenCount() == 1 else 0
                            atom_0_OH0[index] =1 if neighbour_atom.ExplicitHydrogenCount() == 0 else 0

                        atom_1_CH3[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() == 3 else 0
                        atom_1_CH2[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() == 2 else 0
                        atom_1_CH1[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() == 1 else 0
                        atom_1_CH0[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() == 0 else 0
            elif atom.atomicnum == 7:
                if neighbour_atom.GetAtomicNum() == 1:
                    if bond.IsAromatic():
                        atom_0_NH1[index] = 1

                        atom_1_NH1[index] = 1
                    else:
                        atom_0_NH2[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() >= 2 else 0 # > handle NH3
                        atom_0_NH1[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() == 1 else 0
                        atom_0_NH0[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() == 0 else 0

                        atom_1_NH2[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() >= 2 else 0 # > handle NH3
                        atom_1_NH1[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() == 1 else 0
                        atom_1_NH0[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() == 0 else 0
                else:
                    if bond.IsAromatic():
                        if neighbour_atom.GetAtomicNum() == 6:
                            atom_0_CH1[index] = 1

                        elif neighbour_atom.GetAtomicNum() == 7:
                            atom_0_NH1[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 1 else 0
                            atom_0_NH0[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 0 else 0

                        elif neighbour_atom.GetAtomicNum() == 8:
                            atom_0_OH0[index] = 1

                        atom_1_NH1[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() == 1 else 0
                        atom_1_NH0[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() == 0 else 0
                    else:
                        if neighbour_atom.GetAtomicNum() == 6:
                            atom_0_CH3[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 3 else 0
                            atom_0_CH2[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 2 else 0
                            atom_0_CH1[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 1 else 0
                            atom_0_CH0[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 0 else 0
                        elif neighbour_atom.GetAtomicNum() == 7:
                            atom_0_NH2[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 2 else 0
                            atom_0_NH1[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 1 else 0
                            atom_0_NH0[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 0 else 0
                        elif neighbour_atom.GetAtomicNum() == 8:
                            atom_0_OH1[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 1 else 0
                            atom_0_OH0[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 0 else 0

                        atom_1_NH2[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() == 2 else 0
                        atom_1_NH1[index] =1 if atom.OBAtom.ExplicitHydrogenCount() == 1 else 0
                        atom_1_NH0[index] =1 if atom.OBAtom.ExplicitHydrogenCount() == 0 else 0

            elif atom.atomicnum == 8:
                if neighbour_atom.GetAtomicNum() == 1:
                    atom_0_OH1[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() == 1 else 0
                    atom_0_OH0[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() == 0 else 0

                    atom_1_OH1[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() == 1 else 0
                    atom_1_OH0[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() == 0 else 0
                else:
                    if bond.IsAromatic():
                        if neighbour_atom.GetAtomicNum() == 6:
                            atom_0_CH1[index] = 1

                        elif neighbour_atom.GetAtomicNum() == 7:
                            atom_0_NH1[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 1 else 0
                            atom_0_NH0[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 0 else 0

                        elif neighbour_atom.GetAtomicNum() == 8:
                            atom_0_OH0[index] = 1

                        atom_1_OH0[index] = 1
                    else:
                        if neighbour_atom.GetAtomicNum() == 6:
                            atom_0_CH3[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 3 else 0
                            atom_0_CH2[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 2 else 0
                            atom_0_CH1[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 1 else 0
                            atom_0_CH0[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 0 else 0
                        elif neighbour_atom.GetAtomicNum() == 7:
                            atom_0_NH2[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 2 else 0
                            atom_0_NH1[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 1 else 0
                            atom_0_NH0[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 0 else 0
                        elif neighbour_atom.GetAtomicNum() == 8:
                            atom_0_OH1[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 1 else 0
                            atom_0_OH0[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 0 else 0

                        atom_1_OH1[index] =1 if atom.OBAtom.ExplicitHydrogenCount() == 1 else 0
                        atom_1_OH0[index] =1 if atom.OBAtom.ExplicitHydrogenCount() == 0 else 0

            is_in_ring[index] = 1 if bond.IsInRing() else 0
            has_aromatic_bond[index] = 1 if bond.IsAromatic() else 0
            is_primary_amide[index] = 1 if bond.IsPrimaryAmide() else 0
            is_secondary_amide[index] = 1 if bond.IsSecondaryAmide() else 0
            is_tertiary_amide[index] = 1 if bond.IsTertiaryAmide() else 0
            is_ester[index] = 1 if bond.IsEster() else 0
            is_carbonyl[index] = 1 if bond.IsCarbonyl() else 0

            bond_order[index] = bond.GetBO()
            bond_length[index] = bond.GetLength()
            
            print("      Bond {}, length {}, order {}".format(bond.GetIdx(), bond.GetLength(), bond.GetBO()))
    
    mol_attributes_map = {'mol_name' : mol_name,
                          'atom_0' : atom_0,
                          'atom_1' : atom_1,
                          'atom_0_CH3' : atom_0_CH3,
                          'atom_1_CH3' : atom_1_CH3,
                          'atom_0_CH2' : atom_0_CH2,
                          'atom_1_CH2' : atom_1_CH2,
                          'atom_0_CH1' : atom_0_CH1,
                          'atom_1_CH1' : atom_1_CH1,
                          'atom_0_CH0' : atom_0_CH0,
                          'atom_1_CH0' : atom_1_CH0,
                          'atom_0_NH2' : atom_0_NH2,
                          'atom_1_NH2' : atom_1_NH2,
                          'atom_0_NH1' : atom_0_NH1,
                          'atom_1_NH1' : atom_1_NH1,
                          'atom_0_NH0' : atom_0_NH0,
                          'atom_1_NH0' : atom_1_NH0,
                          'atom_0_OH1' : atom_0_OH1,
                          'atom_1_OH1' : atom_1_OH1,
                          'atom_0_OH0' : atom_0_OH0,
                          'atom_1_OH0' : atom_1_OH0,
                          'is_in_ring' : is_in_ring,
                          'has_aromatic_bond' : has_aromatic_bond,
                          'is_primary_amide' : is_primary_amide,
                          'is_secondary_amide' : is_secondary_amide,
                          'is_tertiary_amide' : is_tertiary_amide,
                          'is_ester' : is_ester,
                          'is_carbonyl' : is_carbonyl,
                          'bond_order' : bond_order,
                          'bond_length' : bond_length,
    }
    return mol_attributes_map
        

In [11]:
import pandas

pandas.set_option('display.max_columns', None)

def create_molecule_df(mol_attributes_map):
    df = pd.DataFrame({'molecule_name': mol_attributes_map['mol_name'],
                        'atom_0': mol_attributes_map['atom_0'],
                        'atom_1': mol_attributes_map['atom_1'],
                        'atom_0_CH3': mol_attributes_map['atom_0_CH3'],
                        'atom_1_CH3': mol_attributes_map['atom_1_CH3'],
                        'atom_0_CH2': mol_attributes_map['atom_0_CH2'],
                        'atom_1_CH2': mol_attributes_map['atom_1_CH2'],
                        'atom_0_CH1': mol_attributes_map['atom_0_CH1'],
                        'atom_1_CH1': mol_attributes_map['atom_1_CH1'],
                        'atom_0_CH0': mol_attributes_map['atom_0_CH0'],
                        'atom_1_CH0': mol_attributes_map['atom_1_CH0'],

                        'atom_0_NH2': mol_attributes_map['atom_0_NH2'],
                        'atom_1_NH2': mol_attributes_map['atom_1_NH2'],
                        'atom_0_NH1': mol_attributes_map['atom_0_NH1'],
                        'atom_1_NH1': mol_attributes_map['atom_1_NH1'],
                        'atom_0_NH0': mol_attributes_map['atom_0_NH0'],
                        'atom_1_NH0': mol_attributes_map['atom_1_NH0'],

                        'atom_0_OH1': mol_attributes_map['atom_0_OH1'],
                        'atom_1_OH1': mol_attributes_map['atom_1_OH1'],
                        'atom_0_OH0': mol_attributes_map['atom_0_OH0'],
                        'atom_1_OH0': mol_attributes_map['atom_1_OH0'],

                        'is_in_ring': mol_attributes_map['is_in_ring'],
                        'has_aromatic_bond': mol_attributes_map['has_aromatic_bond'],                  
                        'is_primary_amide': mol_attributes_map['is_primary_amide'],
                        'is_secondary_amide': mol_attributes_map['is_secondary_amide'],
                        'is_tertiary_amide': mol_attributes_map['is_tertiary_amide'],
                        'is_ester': mol_attributes_map['is_ester'],
                        'is_carbonyl': mol_attributes_map['is_carbonyl'],

                        'bond_order': mol_attributes_map['bond_order'],
                        'bond_length': mol_attributes_map['bond_length'],
                      })
    return df


In [14]:
%%timeit
dfs = []
mol_attributes_map = describe_molecule('dsgdb9nsd_000010')
test_df1 = create_molecule_df(mol_attributes_map)
dfs.append(test_df1)

mol_attributes_map = describe_molecule('dsgdb9nsd_000013')
test_df2 = create_molecule_df(mol_attributes_map)
dfs.append(test_df2)

mol_attributes_map = describe_molecule('dsgdb9nsd_000050')
test_df3 = create_molecule_df(mol_attributes_map)
dfs.append(test_df3)

mol_attributes_map = describe_molecule('dsgdb9nsd_010000')
test_df4 = create_molecule_df(mol_attributes_map)
dfs.append(test_df4)

mol_attributes_map = describe_molecule('dsgdb9nsd_040000')
test_df5 = create_molecule_df(mol_attributes_map)
dfs.append(test_df5)

merged = pd.concat(dfs)

#test_df.head(10)

Molecule length: 5
Atom index 1
Index: 0
  atom index:atom 1: 6, Neighbor index:atom 5: 1
      Bond 0, length 1.0933192240105922, order 1
Index: 1
  atom index:atom 1: 6, Neighbor index:atom 2: 6
      Bond 2, length 1.4569296189663543, order 1
Index: 2
  atom index:atom 1: 6, Neighbor index:atom 4: 1
      Bond 3, length 1.0933172562791795, order 1
Index: 3
  atom index:atom 1: 6, Neighbor index:atom 6: 1
      Bond 4, length 1.093305764923213, order 1
Atom index 2
Index: 4
  atom index:atom 2: 6, Neighbor index:atom 3: 7
      Bond 1, length 1.1550085723365782, order 3
Atom index 3
Molecule length: 10
Atom index 1
Index: 0
  atom index:atom 1: 6, Neighbor index:atom 5: 1
      Bond 4, length 1.0959484787544758, order 1
Index: 1
  atom index:atom 1: 6, Neighbor index:atom 2: 6
      Bond 6, length 1.5312569224780486, order 1
Index: 2
  atom index:atom 1: 6, Neighbor index:atom 4: 1
      Bond 8, length 1.09594576644171, order 1
Index: 3
  atom index:atom 1: 6, Neighbor index:atom 6: 

Molecule length: 10
Atom index 1
Index: 0
  atom index:atom 1: 7, Neighbor index:atom 2: 6
      Bond 6, length 1.3720275651673, order 1
Index: 1
  atom index:atom 1: 7, Neighbor index:atom 5: 6
      Bond 8, length 1.3720269177819915, order 1
Index: 2
  atom index:atom 1: 7, Neighbor index:atom 6: 1
      Bond 9, length 1.0044476267664588, order 1
Atom index 2
Index: 3
  atom index:atom 2: 6, Neighbor index:atom 3: 6
      Bond 3, length 1.3754011801052222, order 2
Index: 4
  atom index:atom 2: 6, Neighbor index:atom 7: 1
      Bond 5, length 1.0781095567933316, order 1
Atom index 3
Index: 5
  atom index:atom 3: 6, Neighbor index:atom 8: 1
      Bond 0, length 1.0791686712383692, order 1
Index: 6
  atom index:atom 3: 6, Neighbor index:atom 4: 6
      Bond 2, length 1.4235445979523695, order 1
Atom index 4
Index: 7
  atom index:atom 4: 6, Neighbor index:atom 9: 1
      Bond 1, length 1.0791690294690102, order 1
Index: 8
  atom index:atom 4: 6, Neighbor index:atom 5: 6
      Bond 4, len

Molecule length: 10
Atom index 1
Index: 0
  atom index:atom 1: 6, Neighbor index:atom 5: 1
      Bond 4, length 1.0959484787544758, order 1
Index: 1
  atom index:atom 1: 6, Neighbor index:atom 2: 6
      Bond 6, length 1.5312569224780486, order 1
Index: 2
  atom index:atom 1: 6, Neighbor index:atom 4: 1
      Bond 8, length 1.09594576644171, order 1
Index: 3
  atom index:atom 1: 6, Neighbor index:atom 6: 1
      Bond 9, length 1.0946596356461136, order 1
Atom index 2
Index: 4
  atom index:atom 2: 6, Neighbor index:atom 3: 6
      Bond 3, length 1.5312568231773664, order 1
Index: 5
  atom index:atom 2: 6, Neighbor index:atom 8: 1
      Bond 5, length 1.0972659014514141, order 1
Index: 6
  atom index:atom 2: 6, Neighbor index:atom 7: 1
      Bond 7, length 1.0972661196148408, order 1
Atom index 3
Index: 7
  atom index:atom 3: 6, Neighbor index:atom 11: 1
      Bond 0, length 1.0959458045438788, order 1
Index: 8
  atom index:atom 3: 6, Neighbor index:atom 9: 1
      Bond 1, length 1.09594

Molecule length: 10
Atom index 1
Index: 0
  atom index:atom 1: 7, Neighbor index:atom 2: 6
      Bond 6, length 1.3720275651673, order 1
Index: 1
  atom index:atom 1: 7, Neighbor index:atom 5: 6
      Bond 8, length 1.3720269177819915, order 1
Index: 2
  atom index:atom 1: 7, Neighbor index:atom 6: 1
      Bond 9, length 1.0044476267664588, order 1
Atom index 2
Index: 3
  atom index:atom 2: 6, Neighbor index:atom 3: 6
      Bond 3, length 1.3754011801052222, order 2
Index: 4
  atom index:atom 2: 6, Neighbor index:atom 7: 1
      Bond 5, length 1.0781095567933316, order 1
Atom index 3
Index: 5
  atom index:atom 3: 6, Neighbor index:atom 8: 1
      Bond 0, length 1.0791686712383692, order 1
Index: 6
  atom index:atom 3: 6, Neighbor index:atom 4: 6
      Bond 2, length 1.4235445979523695, order 1
Atom index 4
Index: 7
  atom index:atom 4: 6, Neighbor index:atom 9: 1
      Bond 1, length 1.0791690294690102, order 1
Index: 8
  atom index:atom 4: 6, Neighbor index:atom 5: 6
      Bond 4, len

Molecule length: 25
Atom index 1
Index: 0
  atom index:atom 1: 6, Neighbor index:atom 11: 1
      Bond 8, length 1.0855124880166622, order 1
Index: 1
  atom index:atom 1: 6, Neighbor index:atom 2: 6
      Bond 10, length 1.5086579726913323, order 1
Index: 2
  atom index:atom 1: 6, Neighbor index:atom 3: 6
      Bond 13, length 1.5153135110307625, order 1
Index: 3
  atom index:atom 1: 6, Neighbor index:atom 10: 1
      Bond 17, length 1.0855124882958451, order 1
Atom index 2
Index: 4
  atom index:atom 2: 6, Neighbor index:atom 12: 1
      Bond 7, length 1.0856435373779914, order 1
Index: 5
  atom index:atom 2: 6, Neighbor index:atom 3: 6
      Bond 9, length 1.5054268169131797, order 1
Index: 6
  atom index:atom 2: 6, Neighbor index:atom 13: 1
      Bond 11, length 1.085643531511439, order 1
Atom index 3
Index: 7
  atom index:atom 3: 6, Neighbor index:atom 9: 6
      Bond 4, length 1.5333136074519638, order 1
Index: 8
  atom index:atom 3: 6, Neighbor index:atom 4: 6
      Bond 14, lengt

...

Molecule length: 10
Atom index 1
Index: 0
  atom index:atom 1: 8, Neighbor index:atom 2: 6
      Bond 4, length 1.186773737292343, order 2
Atom index 2
Index: 1
  atom index:atom 2: 6, Neighbor index:atom 3: 8
      Bond 3, length 1.3659913286511252, order 1
Index: 2
  atom index:atom 2: 6, Neighbor index:atom 9: 1
      Bond 7, length 1.1056295908468485, order 1
Atom index 3
Index: 3
  atom index:atom 3: 8, Neighbor index:atom 4: 6
      Bond 2, length 1.4227054000340489, order 1
Atom index 4
Index: 4
  atom index:atom 4: 6, Neighbor index:atom 7: 6
      Bond 1, length 1.4678346406212508, order 1
Index: 5
  atom index:atom 4: 6, Neighbor index:atom 10: 1
      Bond 5, length 1.0967549698571413, order 1
Index: 6
  atom index:atom 4: 6, Neighbor index:atom 5: 6
      Bond 6, length 1.5501134485989634, order 1
Atom index 5
Index: 7
  atom index:atom 5: 6, Neighbor index:atom 11: 1
      Bond 8, length 1.1074767108023804, order 1
Index: 8
  atom index:atom 5: 6, Neighbor index:atom 6: 8


Molecule length: 10
Atom index 1
Index: 0
  atom index:atom 1: 6, Neighbor index:atom 5: 1
      Bond 4, length 1.0959484787544758, order 1
Index: 1
  atom index:atom 1: 6, Neighbor index:atom 2: 6
      Bond 6, length 1.5312569224780486, order 1
Index: 2
  atom index:atom 1: 6, Neighbor index:atom 4: 1
      Bond 8, length 1.09594576644171, order 1
Index: 3
  atom index:atom 1: 6, Neighbor index:atom 6: 1
      Bond 9, length 1.0946596356461136, order 1
Atom index 2
Index: 4
  atom index:atom 2: 6, Neighbor index:atom 3: 6
      Bond 3, length 1.5312568231773664, order 1
Index: 5
  atom index:atom 2: 6, Neighbor index:atom 8: 1
      Bond 5, length 1.0972659014514141, order 1
Index: 6
  atom index:atom 2: 6, Neighbor index:atom 7: 1
      Bond 7, length 1.0972661196148408, order 1
Atom index 3
Index: 7
  atom index:atom 3: 6, Neighbor index:atom 11: 1
      Bond 0, length 1.0959458045438788, order 1
Index: 8
  atom index:atom 3: 6, Neighbor index:atom 9: 1
      Bond 1, length 1.09594

      Bond 0, length 1.1550643625108126, order 3
Atom index 8
Molecule length: 25
Atom index 1
Index: 0
  atom index:atom 1: 6, Neighbor index:atom 11: 1
      Bond 8, length 1.0855124880166622, order 1
Index: 1
  atom index:atom 1: 6, Neighbor index:atom 2: 6
      Bond 10, length 1.5086579726913323, order 1
Index: 2
  atom index:atom 1: 6, Neighbor index:atom 3: 6
      Bond 13, length 1.5153135110307625, order 1
Index: 3
  atom index:atom 1: 6, Neighbor index:atom 10: 1
      Bond 17, length 1.0855124882958451, order 1
Atom index 2
Index: 4
  atom index:atom 2: 6, Neighbor index:atom 12: 1
      Bond 7, length 1.0856435373779914, order 1
Index: 5
  atom index:atom 2: 6, Neighbor index:atom 3: 6
      Bond 9, length 1.5054268169131797, order 1
Index: 6
  atom index:atom 2: 6, Neighbor index:atom 13: 1
      Bond 11, length 1.085643531511439, order 1
Atom index 3
Index: 7
  atom index:atom 3: 6, Neighbor index:atom 9: 6
      Bond 4, length 1.5333136074519638, order 1
Index: 8
  atom 

Molecule length: 25
Atom index 1
Index: 0
  atom index:atom 1: 6, Neighbor index:atom 11: 1
      Bond 8, length 1.0855124880166622, order 1
Index: 1
  atom index:atom 1: 6, Neighbor index:atom 2: 6
      Bond 10, length 1.5086579726913323, order 1
Index: 2
  atom index:atom 1: 6, Neighbor index:atom 3: 6
      Bond 13, length 1.5153135110307625, order 1
Index: 3
  atom index:atom 1: 6, Neighbor index:atom 10: 1
      Bond 17, length 1.0855124882958451, order 1
Atom index 2
Index: 4
  atom index:atom 2: 6, Neighbor index:atom 12: 1
      Bond 7, length 1.0856435373779914, order 1
Index: 5
  atom index:atom 2: 6, Neighbor index:atom 3: 6
      Bond 9, length 1.5054268169131797, order 1
Index: 6
  atom index:atom 2: 6, Neighbor index:atom 13: 1
      Bond 11, length 1.085643531511439, order 1
Atom index 3
Index: 7
  atom index:atom 3: 6, Neighbor index:atom 9: 6
      Bond 4, length 1.5333136074519638, order 1
Index: 8
  atom index:atom 3: 6, Neighbor index:atom 4: 6
      Bond 14, lengt

49.1 ms ± 999 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
merged.head(200)

,molecule_name,atom_0,atom_1,atom_0_CH3,atom_1_CH3,atom_0_CH2,atom_1_CH2,atom_0_CH1,atom_1_CH1,atom_0_CH0,atom_1_CH0,atom_0_NH2,atom_1_NH2,atom_0_NH1,atom_1_NH1,atom_0_NH0,atom_1_NH0,atom_0_OH1,atom_1_OH1,atom_0_OH0,atom_1_OH0,is_in_ring,has_aromatic_bond,is_primary_amide,is_secondary_amide,is_tertiary_amide,is_ester,is_carbonyl,bond_order,bond_length
0,dsgdb9nsd_000010,4,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.093319
1,dsgdb9nsd_000010,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.456930
2,dsgdb9nsd_000010,3,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.093317
3,dsgdb9nsd_000010,5,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.093306
4,dsgdb9nsd_000010,2,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,3,1.155009
0,dsgdb9nsd_000013,4,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.095948
1,dsgdb9nsd_000013,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.531257
2,dsgdb9nsd_000013,3,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.095946
3,dsgdb9nsd_000013,5,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.094660
4,dsgdb9nsd_000013,2,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.531257


### So far the code has only considered bonds between neighbour atoms. How do we find and represent all 2nd and 3rd class neighbours? 

In [41]:
import openbabel

def describe_molecule_neighbours(name):
    
    mol = next(pybel.readfile("xyz", "./input/structures/" + name + ".xyz"))
    mol_length = mol.OBMol.NumBonds()

    print(f'Molecule length: {mol_length}')
    # Define containers
    
    
    mol_name = [0] * mol_length # This will be our dataframe index
    atom_0 = [0] * mol_length
    atom_1 = [0] * mol_length
    
    atom_1_1_level_NB = [0] * mol_length
    atom_1_2_level_NB = [0] * mol_length
    
    bond_order = [0] * mol_length
    bond_length = [0] * mol_length
    is_in_ring = [0] * mol_length
    has_aromatic_bond = [0] * mol_length
    is_primary_amide = [0] * mol_length
    is_secondary_amide = [0] * mol_length
    is_tertiary_amide = [0] * mol_length
    is_ester = [0] * mol_length
    is_carbonyl = [0] * mol_length

    atom_0_CH3 = [0] * mol_length
    atom_0_CH2= [0] * mol_length
    atom_0_CH1= [0] * mol_length
    atom_0_CH0= [0] * mol_length
    atom_1_CH3 = [0] * mol_length
    atom_1_CH2= [0] * mol_length
    atom_1_CH1= [0] * mol_length
    atom_1_CH0= [0] * mol_length

    atom_0_NH2 = [0] * mol_length
    atom_0_NH1= [0] * mol_length
    atom_0_NH0= [0] * mol_length
    atom_1_NH2 = [0] * mol_length
    atom_1_NH1= [0] * mol_length
    atom_1_NH0= [0] * mol_length

    atom_0_OH1 = [0] * mol_length
    atom_0_OH0= [0] * mol_length
    atom_1_OH1 = [0] * mol_length
    atom_1_OH0= [0] * mol_length

    bonds_considered = {}
    index = -1
    for atom in mol.atoms:

        if atom.atomicnum == 1: # ignore H atoms
            continue

        atom_id = atom.idx
        #print("Atom index {}".format(atom_id))

        level_1_atoms = set()
        level_2_map = dict()        
        for NB_level_1_atom in openbabel.OBAtomAtomIter(atom.OBAtom):
            if NB_level_1_atom.GetAtomicNum() != 1:
                level_1_atoms.add(NB_level_1_atom.GetIdx()-1)
                level_2_atoms = set()
                for NB_level_2_atom in openbabel.OBAtomAtomIter(NB_level_1_atom):
                    if NB_level_2_atom.GetAtomicNum() != 1:
                        level_2_atoms.add(NB_level_2_atom.GetIdx()-1)
                level_2_map[NB_level_1_atom.GetIdx()-1] = level_2_atoms
        
        for neighbour_atom in openbabel.OBAtomAtomIter(atom.OBAtom):

            neighbour_atom_id = neighbour_atom.GetIdx()

            key = str(min(atom_id, neighbour_atom_id)) + '_' + str(max(atom_id, neighbour_atom_id))
            if key in bonds_considered:
                continue
            else:
                bonds_considered[key] = 'counted'

            index += 1
            #print(f'Index: {index}')
                        
            #print("  atom index:atom {}: {}, Neighbor index:atom {}: {}".format(atom_id, atom.atomicnum, neighbour_atom_id, neighbour_atom.GetAtomicNum()))
            bond = mol.OBMol.GetBond(atom.idx, neighbour_atom.GetIdx())

            mol_name[index] = name
            atom_0[index] = neighbour_atom_id - 1
            atom_1[index] = atom_id - 1
            atom_1_1_level_NB[index] = level_1_atoms
            atom_1_2_level_NB[index] = level_2_map

            if atom.atomicnum == 6:
                if neighbour_atom.GetAtomicNum() == 1:
                    if bond.IsAromatic():
                        atom_0_CH1[index] = 1

                        atom_1_CH1[index] = 1
                    else:
                        atom_0_CH3[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() >= 3 else 0 # > handle CH4
                        atom_0_CH2[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() == 2 else 0
                        atom_0_CH1[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() == 1 else 0
                        atom_0_CH0[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() == 0 else 0

                        atom_1_CH3[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() >= 3 else 0 # > handle CH4
                        atom_1_CH2[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() == 2 else 0
                        atom_1_CH1[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() == 1 else 0
                        atom_1_CH0[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() == 0 else 0
                else:
                    if bond.IsAromatic():
                        if neighbour_atom.GetAtomicNum() == 6:
                            atom_0_CH1[index] = 1

                        elif neighbour_atom.GetAtomicNum() == 7:
                            atom_0_NH1[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 1 else 0
                            atom_0_NH0[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 0 else 0

                        elif neighbour_atom.GetAtomicNum() == 8:
                            atom_0_OH0[index] =1

                        atom_1_CH1[index] = 1
                    else:
                        if neighbour_atom.GetAtomicNum() == 6:
                            atom_0_CH3[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 3 else 0
                            atom_0_CH2[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 2 else 0
                            atom_0_CH1[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 1 else 0
                            atom_0_CH0[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 0 else 0
                        elif neighbour_atom.GetAtomicNum() == 7:
                            atom_0_NH2[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 2 else 0
                            atom_0_NH1[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 1 else 0
                            atom_0_NH0[index] =1 if neighbour_atom.ExplicitHydrogenCount() == 0 else 0
                        elif neighbour_atom.GetAtomicNum() == 8:
                            atom_0_OH1[index] =1 if neighbour_atom.ExplicitHydrogenCount() == 1 else 0
                            atom_0_OH0[index] =1 if neighbour_atom.ExplicitHydrogenCount() == 0 else 0

                        atom_1_CH3[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() == 3 else 0
                        atom_1_CH2[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() == 2 else 0
                        atom_1_CH1[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() == 1 else 0
                        atom_1_CH0[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() == 0 else 0
            elif atom.atomicnum == 7:
                if neighbour_atom.GetAtomicNum() == 1:
                    if bond.IsAromatic():
                        atom_0_NH1[index] = 1

                        atom_1_NH1[index] = 1
                    else:
                        atom_0_NH2[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() >= 2 else 0 # > handle NH3
                        atom_0_NH1[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() == 1 else 0
                        atom_0_NH0[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() == 0 else 0

                        atom_1_NH2[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() >= 2 else 0 # > handle NH3
                        atom_1_NH1[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() == 1 else 0
                        atom_1_NH0[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() == 0 else 0
                else:
                    if bond.IsAromatic():
                        if neighbour_atom.GetAtomicNum() == 6:
                            atom_0_CH1[index] = 1

                        elif neighbour_atom.GetAtomicNum() == 7:
                            atom_0_NH1[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 1 else 0
                            atom_0_NH0[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 0 else 0

                        elif neighbour_atom.GetAtomicNum() == 8:
                            atom_0_OH0[index] = 1

                        atom_1_NH1[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() == 1 else 0
                        atom_1_NH0[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() == 0 else 0
                    else:
                        if neighbour_atom.GetAtomicNum() == 6:
                            atom_0_CH3[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 3 else 0
                            atom_0_CH2[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 2 else 0
                            atom_0_CH1[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 1 else 0
                            atom_0_CH0[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 0 else 0
                        elif neighbour_atom.GetAtomicNum() == 7:
                            atom_0_NH2[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 2 else 0
                            atom_0_NH1[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 1 else 0
                            atom_0_NH0[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 0 else 0
                        elif neighbour_atom.GetAtomicNum() == 8:
                            atom_0_OH1[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 1 else 0
                            atom_0_OH0[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 0 else 0

                        atom_1_NH2[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() == 2 else 0
                        atom_1_NH1[index] =1 if atom.OBAtom.ExplicitHydrogenCount() == 1 else 0
                        atom_1_NH0[index] =1 if atom.OBAtom.ExplicitHydrogenCount() == 0 else 0

            elif atom.atomicnum == 8:
                if neighbour_atom.GetAtomicNum() == 1:
                    atom_0_OH1[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() == 1 else 0
                    atom_0_OH0[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() == 0 else 0

                    atom_1_OH1[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() == 1 else 0
                    atom_1_OH0[index] = 1 if atom.OBAtom.ExplicitHydrogenCount() == 0 else 0
                else:
                    if bond.IsAromatic():
                        if neighbour_atom.GetAtomicNum() == 6:
                            atom_0_CH1[index] = 1

                        elif neighbour_atom.GetAtomicNum() == 7:
                            atom_0_NH1[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 1 else 0
                            atom_0_NH0[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 0 else 0

                        elif neighbour_atom.GetAtomicNum() == 8:
                            atom_0_OH0[index] = 1

                        atom_1_OH0[index] = 1
                    else:
                        if neighbour_atom.GetAtomicNum() == 6:
                            atom_0_CH3[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 3 else 0
                            atom_0_CH2[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 2 else 0
                            atom_0_CH1[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 1 else 0
                            atom_0_CH0[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 0 else 0
                        elif neighbour_atom.GetAtomicNum() == 7:
                            atom_0_NH2[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 2 else 0
                            atom_0_NH1[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 1 else 0
                            atom_0_NH0[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 0 else 0
                        elif neighbour_atom.GetAtomicNum() == 8:
                            atom_0_OH1[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 1 else 0
                            atom_0_OH0[index] = 1 if neighbour_atom.ExplicitHydrogenCount() == 0 else 0

                        atom_1_OH1[index] =1 if atom.OBAtom.ExplicitHydrogenCount() == 1 else 0
                        atom_1_OH0[index] =1 if atom.OBAtom.ExplicitHydrogenCount() == 0 else 0

            is_in_ring[index] = 1 if bond.IsInRing() else 0
            has_aromatic_bond[index] = 1 if bond.IsAromatic() else 0
            is_primary_amide[index] = 1 if bond.IsPrimaryAmide() else 0
            is_secondary_amide[index] = 1 if bond.IsSecondaryAmide() else 0
            is_tertiary_amide[index] = 1 if bond.IsTertiaryAmide() else 0
            is_ester[index] = 1 if bond.IsEster() else 0
            is_carbonyl[index] = 1 if bond.IsCarbonyl() else 0

            bond_order[index] = bond.GetBO()
            bond_length[index] = bond.GetLength()
            
            #print("      Bond {}, length {}, order {}".format(bond.GetIdx(), bond.GetLength(), bond.GetBO()))
    
    mol_attributes_map = {'mol_name' : mol_name,
                          'atom_0' : atom_0,
                          'atom_1' : atom_1,
                          'atom_1_1_level_NB' : atom_1_1_level_NB,
                          'atom_1_2_level_NB' : atom_1_2_level_NB,
                          'atom_0_CH3' : atom_0_CH3,
                          'atom_1_CH3' : atom_1_CH3,
                          'atom_0_CH2' : atom_0_CH2,
                          'atom_1_CH2' : atom_1_CH2,
                          'atom_0_CH1' : atom_0_CH1,
                          'atom_1_CH1' : atom_1_CH1,
                          'atom_0_CH0' : atom_0_CH0,
                          'atom_1_CH0' : atom_1_CH0,
                          'atom_0_NH2' : atom_0_NH2,
                          'atom_1_NH2' : atom_1_NH2,
                          'atom_0_NH1' : atom_0_NH1,
                          'atom_1_NH1' : atom_1_NH1,
                          'atom_0_NH0' : atom_0_NH0,
                          'atom_1_NH0' : atom_1_NH0,
                          'atom_0_OH1' : atom_0_OH1,
                          'atom_1_OH1' : atom_1_OH1,
                          'atom_0_OH0' : atom_0_OH0,
                          'atom_1_OH0' : atom_1_OH0,
                          'is_in_ring' : is_in_ring,
                          'has_aromatic_bond' : has_aromatic_bond,
                          'is_primary_amide' : is_primary_amide,
                          'is_secondary_amide' : is_secondary_amide,
                          'is_tertiary_amide' : is_tertiary_amide,
                          'is_ester' : is_ester,
                          'is_carbonyl' : is_carbonyl,
                          'bond_order' : bond_order,
                          'bond_length' : bond_length,
    }
    return mol_attributes_map
        

In [42]:
import pandas

pandas.set_option('display.max_columns', None)

def create_molecule_neighbours_df(mol_attributes_map):
    df = pd.DataFrame({'molecule_name': mol_attributes_map['mol_name'],
                        'atom_0': mol_attributes_map['atom_0'],
                        'atom_1': mol_attributes_map['atom_1'],
                        'atom_1_1_level_NB' : mol_attributes_map['atom_1_1_level_NB'],
                        'atom_1_2_level_NB' : mol_attributes_map['atom_1_2_level_NB'],
                        'atom_0_CH3': mol_attributes_map['atom_0_CH3'],
                        'atom_1_CH3': mol_attributes_map['atom_1_CH3'],
                        'atom_0_CH2': mol_attributes_map['atom_0_CH2'],
                        'atom_1_CH2': mol_attributes_map['atom_1_CH2'],
                        'atom_0_CH1': mol_attributes_map['atom_0_CH1'],
                        'atom_1_CH1': mol_attributes_map['atom_1_CH1'],
                        'atom_0_CH0': mol_attributes_map['atom_0_CH0'],
                        'atom_1_CH0': mol_attributes_map['atom_1_CH0'],

                        'atom_0_NH2': mol_attributes_map['atom_0_NH2'],
                        'atom_1_NH2': mol_attributes_map['atom_1_NH2'],
                        'atom_0_NH1': mol_attributes_map['atom_0_NH1'],
                        'atom_1_NH1': mol_attributes_map['atom_1_NH1'],
                        'atom_0_NH0': mol_attributes_map['atom_0_NH0'],
                        'atom_1_NH0': mol_attributes_map['atom_1_NH0'],

                        'atom_0_OH1': mol_attributes_map['atom_0_OH1'],
                        'atom_1_OH1': mol_attributes_map['atom_1_OH1'],
                        'atom_0_OH0': mol_attributes_map['atom_0_OH0'],
                        'atom_1_OH0': mol_attributes_map['atom_1_OH0'],

                        'is_in_ring': mol_attributes_map['is_in_ring'],
                        'has_aromatic_bond': mol_attributes_map['has_aromatic_bond'],                  
                        'is_primary_amide': mol_attributes_map['is_primary_amide'],
                        'is_secondary_amide': mol_attributes_map['is_secondary_amide'],
                        'is_tertiary_amide': mol_attributes_map['is_tertiary_amide'],
                        'is_ester': mol_attributes_map['is_ester'],
                        'is_carbonyl': mol_attributes_map['is_carbonyl'],

                        'bond_order': mol_attributes_map['bond_order'],
                        'bond_length': mol_attributes_map['bond_length'],
                      })
    return df


In [45]:
%%timeit
mol_neighbour_attributes_map = describe_molecule_neighbours('dsgdb9nsd_040000')
neighbour_df = create_molecule_neighbours_df(mol_neighbour_attributes_map)

Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25


Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25
Molecule length: 25


In [44]:
neighbour_df.head(25)

,molecule_name,atom_0,atom_1,atom_1_1_level_NB,atom_1_2_level_NB,atom_0_CH3,atom_1_CH3,atom_0_CH2,atom_1_CH2,atom_0_CH1,atom_1_CH1,atom_0_CH0,atom_1_CH0,atom_0_NH2,atom_1_NH2,atom_0_NH1,atom_1_NH1,atom_0_NH0,atom_1_NH0,atom_0_OH1,atom_1_OH1,atom_0_OH0,atom_1_OH0,is_in_ring,has_aromatic_bond,is_primary_amide,is_secondary_amide,is_tertiary_amide,is_ester,is_carbonyl,bond_order,bond_length
0,dsgdb9nsd_040000,10,0,"{1, 2}","{1: {0, 2}, 2: {8, 1, 3, 0}}",0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.085512
1,dsgdb9nsd_040000,1,0,"{1, 2}","{1: {0, 2}, 2: {8, 1, 3, 0}}",0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1.508658
2,dsgdb9nsd_040000,2,0,"{1, 2}","{1: {0, 2}, 2: {8, 1, 3, 0}}",0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1.515314
3,dsgdb9nsd_040000,9,0,"{1, 2}","{1: {0, 2}, 2: {8, 1, 3, 0}}",0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.085512
4,dsgdb9nsd_040000,11,1,"{0, 2}","{2: {8, 1, 3, 0}, 0: {1, 2}}",0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.085644
5,dsgdb9nsd_040000,2,1,"{0, 2}","{2: {8, 1, 3, 0}, 0: {1, 2}}",0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1.505427
6,dsgdb9nsd_040000,12,1,"{0, 2}","{2: {8, 1, 3, 0}, 0: {1, 2}}",0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.085644
7,dsgdb9nsd_040000,8,2,"{8, 1, 3, 0}","{8: {2, 6}, 1: {0, 2}, 0: {1, 2}, 3: {2, 4}}",0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1.533314
8,dsgdb9nsd_040000,3,2,"{8, 1, 3, 0}","{8: {2, 6}, 1: {0, 2}, 0: {1, 2}, 3: {2, 4}}",0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1.533314
9,dsgdb9nsd_040000,4,3,"{2, 4}","{2: {8, 1, 3, 0}, 4: {3, 5, 7}}",0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1.539200
